In [2]:
# import matplotlib.pyplot as plt

# # Assuming you have the following dictionaries
# pnr_flight_dict = {123: 'F123', 456: 'F456', 789: 'F789', 987: 'F987'}
# pnr_score_dict = {123: 90, 456: 80, 789: 95, 987: 70}
# rescheduling_score_dict = {(123, 'F123'): 10, (456, 'F456'): 5, (789, 'F789'): 15, (987, 'F987'): 8}

# # Sort the PNRS by scores
# sorted_pnrs = sorted(pnr_score_dict.keys(), key=lambda x: pnr_score_dict[x], reverse=True)

# # Extract flight IDs and scores for each PNR
# flight_ids = [pnr_flight_dict[pnr] for pnr in sorted_pnrs]
# scores = [pnr_score_dict[pnr] for pnr in sorted_pnrs]

# # Plotting
# fig, ax = plt.subplots(figsize=(10, 6))
# bar_width = 0.35
# index = range(len(sorted_pnrs))

# # Plotting the scores
# bars1 = ax.bar(index, scores, bar_width, label='Original Score')

# # Plotting the rescheduling scores as a line plot
# rescheduling_scores = [rescheduling_score_dict[(pnr, pnr_flight_dict[pnr])] for pnr in sorted_pnrs]
# line = ax.plot(index, rescheduling_scores, label='Rescheduling Score', marker='o', color='red')

# # Adding labels and title
# ax.set_xlabel('PNR')
# ax.set_ylabel('Score')
# ax.set_title('PNRs with High Scores and Rescheduling Scores')
# ax.set_xticks(index)
# ax.set_xticklabels(sorted_pnrs)
# ax.legend()

# # Adding flight IDs as text on top of bars
# for i, score in enumerate(scores):
#     ax.text(i, score + 1, pnr_flight_dict[sorted_pnrs[i]], ha='center')

# # Display the plot
# plt.tight_layout()
# plt.show()

In [5]:
# import plotly.graph_objects as go

# # Assuming you have the following dictionaries
# pnr_flight_dict = {123: 'F123', 456: 'F456', 789: 'F789', 987: 'F987'}
# pnr_score_dict = {123: 90, 456: 80, 789: 95, 987: 70}
# rescheduling_score_dict = {(123, 'F123'): 10, (456, 'F456'): 5, (789, 'F789'): 15, (987, 'F987'): 8}

# # Sort the PNRS by scores
# sorted_pnrs = sorted(pnr_score_dict.keys(), key=lambda x: pnr_score_dict[x], reverse=True)

# # Extract flight IDs and scores for each PNR
# flight_ids = [pnr_flight_dict[pnr] for pnr in sorted_pnrs]
# original_scores = [pnr_score_dict[pnr] for pnr in sorted_pnrs]
# rescheduling_scores = [rescheduling_score_dict[(pnr, pnr_flight_dict[pnr])] for pnr in sorted_pnrs]

# # Create a bar chart
# fig = go.Figure()

# # Bar chart for original scores
# fig.add_trace(go.Bar(x=sorted_pnrs, y=original_scores, name='Original Score'))

# # Line chart for rescheduling scores
# fig.add_trace(go.Scatter(x=sorted_pnrs, y=rescheduling_scores, mode='lines+markers', name='Rescheduling Score', line=dict(color='red')))

# # Layout customization
# fig.update_layout(
#     xaxis=dict(title='PNR'),
#     yaxis=dict(title='Score'),
#     title='PNRs with High Scores and Rescheduling Scores',
#     barmode='group'
# )

# # Display the plot
# fig.show()


In [6]:
### TEST PLOTS
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd

In [7]:
df = pd.read_csv('result.csv')
print(df.keys()) 

Index(['pnr', 'pnr_score', 'canclled_flight', 'canclled_class',
       'canclled_flight_departure', 'canclled_flight_arrival', 'canclled_src',
       'canclled_dst', 'allocated_src', 'allocated_dst', 'allocated_flights',
       'allocated_flights_departure', 'allocated_flights_arrival',
       'allocated_classes', 'allocated_flights_score'],
      dtype='object')


In [8]:
# plot for pnr_score distribution of cancelled flights-pnrs
mean_score = df["pnr_score"].mean()
std_dev_score = df["pnr_score"].std()
fig = px.bar(
    df, x="pnr", y="pnr_score", title="PNR Scores", labels={"pnr_score": "PNR Score"}
)
# Add mean and standard deviation annotations
fig.add_shape(
    type="line",
    x0=-0.5,
    x1=len(df) - 0.5,
    y0=mean_score,
    y1=mean_score,
    line=dict(color="red", width=2),
    name="Mean",
)
# Show the chart
print(mean_score, std_dev_score)
# fig.show()

6497.570850202429 4194.77533747481


In [9]:
# plot for class distribution for cancelled flights
fig = px.pie(df, names='allocated_classes', title='Class Distribution for All PNRS')

# plot for class distribution for cancelled flights
fig2 = px.pie(df, names='canclled_class', title='Class Distribution for Cancelled PNRS')
# fig.show()
# fig2.show()



In [10]:
#Pnr vs delay
# Create a scatter plot for PNR vs delay
df.sort_values(by=['allocated_flights_score'], inplace=True)
df['canclled_flight_arrival'] = pd.to_datetime(df['canclled_flight_arrival'])
df['allocated_flights_arrival'] = pd.to_datetime(df['allocated_flights_arrival'])

# Calculate the delay
df['delay'] = (df['allocated_flights_arrival'] - df['canclled_flight_arrival']).dt.total_seconds() / 3600

# Create a scatter plot for PNR vs delay
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=["Scores", "PNR vs Delay", "Pnr Scores"])

# Plot PNR Scores
fig.add_trace(
    px.bar(df, x="pnr", y="allocated_flights_score").update_traces(marker_color='blue').data[0],
    row=1, col=1
)

# Plot PNR vs Delay
fig.add_trace(
    px.scatter(df, x='pnr', y='delay').data[0],
    row=1, col=1
)

fig.add_trace(
    px.scatter(df, x='pnr', y='delay').data[0],
    row=2, col=1
)

fig.add_trace(
    px.bar(df, x="pnr", y="pnr_score").update_traces(marker_color='green').data[0],
    row=2, col=1
)

# Calculate and print the mean delay
mean_delay = df['delay'].mean()
print(f'Mean Delay: {mean_delay} hours, std_dev: {df["delay"].std()}')

# Show the plot
# fig.show()

Mean Delay: 27.613747228381374 hours, std_dev: 21.943406489684325
